In [ ]:
from constants import PATH_JSON_ANNOTATIONS, PATH_JSON_QUESTIONS, OPENAI_KEY, PROMPT_CONSTRUCAO_TEXTO
from utils import load_json, get_image_id, encode_image
import os
import pandas as pd
import openai
from openai import OpenAI
from collections import Counter
import re
import json

In [2]:
json_annotations = load_json(PATH_JSON_ANNOTATIONS)

In [4]:
json_questions = load_json(PATH_JSON_QUESTIONS)

In [5]:
json_annotations, json_questions

({'info': {'description': 'This is v2.0 of the VQA dataset.',
   'url': 'http://visualqa.org',
   'version': '2.0',
   'year': 2017,
   'contributor': 'VQA Team',
   'date_created': '2017-04-26 17:00:44'},
  'license': {'url': 'http://creativecommons.org/licenses/by/4.0/',
   'name': 'Creative Commons Attribution 4.0 International License'},
  'data_subtype': 'val2014',
  'annotations': [{'question_type': 'none of the above',
    'multiple_choice_answer': 'down',
    'answers': [{'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 1},
     {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 2},
     {'answer': 'at table', 'answer_confidence': 'yes', 'answer_id': 3},
     {'answer': 'skateboard', 'answer_confidence': 'yes', 'answer_id': 4},
     {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 5},
     {'answer': 'table', 'answer_confidence': 'yes', 'answer_id': 6},
     {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 7},
     {'answer': 'down', 'an

## Generating Descriptions

In [5]:
dict_df = {'image_path': [], 'image_id': [], 'question': [], 'answer': []}

for i, image in enumerate(os.listdir('images')):
    image_id = get_image_id(image)
    print(f'Imagem {i}, ID: {image_id}')

    for annotation in json_annotations['annotations']:
        if image_id == str(annotation['image_id']):
            answers = annotation['answers']
            image_answers = []
            for answer in answers:
                image_answers.append(answer['answer'])

            chosen_answer = Counter(image_answers).most_common(1)[0][0] # obtendo a resposta que aparece mais vezes para ground-truth    

            question_id = annotation['question_id']
            for question in json_questions['questions']:
                if str(question_id) == str(question['question_id']):
                    chosen_question = question['question']
                    break

            dict_df.get('image_id').append(image_id)
            dict_df.get('question').append(chosen_question)
            dict_df.get('answer').append(chosen_answer)
            dict_df.get('image_path').append(image)
            break

Imagem 0, ID: 42
Imagem 1, ID: 73
Imagem 2, ID: 74
Imagem 3, ID: 133
Imagem 4, ID: 136
Imagem 5, ID: 139
Imagem 6, ID: 143
Imagem 7, ID: 164
Imagem 8, ID: 192
Imagem 9, ID: 196
Imagem 10, ID: 208
Imagem 11, ID: 241
Imagem 12, ID: 257
Imagem 13, ID: 283
Imagem 14, ID: 285
Imagem 15, ID: 294
Imagem 16, ID: 328
Imagem 17, ID: 338
Imagem 18, ID: 357
Imagem 19, ID: 359
Imagem 20, ID: 360
Imagem 21, ID: 387
Imagem 22, ID: 395
Imagem 23, ID: 397
Imagem 24, ID: 400
Imagem 25, ID: 415
Imagem 26, ID: 428
Imagem 27, ID: 459
Imagem 28, ID: 472
Imagem 29, ID: 474
Imagem 30, ID: 486
Imagem 31, ID: 488
Imagem 32, ID: 502
Imagem 33, ID: 520
Imagem 34, ID: 536
Imagem 35, ID: 544
Imagem 36, ID: 564
Imagem 37, ID: 569
Imagem 38, ID: 589
Imagem 39, ID: 590
Imagem 40, ID: 599
Imagem 41, ID: 623
Imagem 42, ID: 626
Imagem 43, ID: 632
Imagem 44, ID: 636
Imagem 45, ID: 641
Imagem 46, ID: 661
Imagem 47, ID: 675
Imagem 48, ID: 692
Imagem 49, ID: 693
Imagem 50, ID: 699
Imagem 51, ID: 711
Imagem 52, ID: 715
Imagem

In [6]:
df = pd.DataFrame(dict_df)

In [7]:
df

,image_path,image_id,question,answer
0,COCO_val2014_000000000042.jpg,42,What color are the gym shoes?,white
1,COCO_val2014_000000000073.jpg,73,What is the license number?,sv-6260
2,COCO_val2014_000000000074.jpg,74,Does this dog have a collar?,no
3,COCO_val2014_000000000133.jpg,133,What color is lamp?,blue
4,COCO_val2014_000000000136.jpg,136,Is this in a museum?,no
...,...,...,...,...
995,COCO_val2014_000000014135.jpg,14135,Is it daytime?,yes
996,COCO_val2014_000000014151.jpg,14151,Is this at the Olympics?,yes
997,COCO_val2014_000000014167.jpg,14167,Is the skateboarder wearing safety gear?,no
998,COCO_val2014_000000014175.jpg,14175,What is sticking up from the fire hydrant?,nothing


In [14]:
model = OpenAI(api_key=OPENAI_KEY)

In [ ]:
generated_texts = []
base_image_path = 'images/'

for i in range(df.shape[0]):

    image_infos = df.iloc[i]
    image_question = image_infos['question']
    image_answer = image_infos['answer']
    image_path = base_image_path + image_infos['image_path']
    
    encoded_image = encode_image(image_path)

    prompt_formatado = PROMPT_CONSTRUCAO_TEXTO.format(question=image_question, answer=image_answer)

    print(f"Gerando textos para a imagem {i}.")

    response = model.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url" : {"url": f"data:image/jpeg;base64,{encoded_image}"}
                    },
                    {
                        "type": "text",
                        "text": prompt_formatado
                    }
                ]
            }
        ], temperature=0.0, timeout=20.0
    )
    texts = response.choices[0].message.content
    generated_texts.append(texts)

    print(f"Textos gerados: {texts}")


In [ ]:
texts_match = []
texts_corrupt = []
answers_text = []

wrongs = []

pattern = r"Description 1:\s*(.*?)Description 2:\s*(.*?)Answer 2:\s*(.*)"
for i, generated_text in enumerate(generated_texts):
    try:
        clean_text = generated_text.replace('\n', '')
        match = re.search(pattern, clean_text)

        if match:
            description1 = match.group(1).strip()
            description2 = match.group(2).strip()
            answer2 = match.group(3).strip()

            texts_match.append(description1)
            texts_corrupt.append(description2)
            answers_text.append(answer2)

        else:
            print(f'Resposta inesperada para a imagem {i}: {clean_text}')
            texts_match.append('empty')
            texts_corrupt.append('empty')
            answers_text.append('empty')
            wrongs.append(generated_text)            

    except Exception as e:
        print(f'Erro ao carregar resposta: {e}')
        wrongs.append(generated_text)

Resposta inesperada para a imagem 118: I'm sorry, I can't assist with that request.
Resposta inesperada para a imagem 144: I'm sorry, I can't assist with that.
Resposta inesperada para a imagem 388: I'm sorry, I can't assist with that.
Resposta inesperada para a imagem 462: I'm sorry, I can't assist with that request.
Resposta inesperada para a imagem 602: I'm sorry, I can't assist with that.
Resposta inesperada para a imagem 635: I'm sorry, I can't assist with that.
Resposta inesperada para a imagem 643: I'm sorry, I can't assist with that.
Resposta inesperada para a imagem 659: I'm sorry, I can't determine the time from the image.
Resposta inesperada para a imagem 667: I'm sorry, I can't assist with that.
Resposta inesperada para a imagem 957: I'm sorry, I can't assist with that.


In [36]:
# atualizando dataframe com as descrições e respostas geradas

df['Tm'] = pd.Series(texts_match)
df['Tc'] = pd.Series(texts_corrupt)
df['Ans'] = pd.Series(answers_text)

In [37]:
df

,image_path,image_id,question,answer,Tm,Tc,Ans
0,COCO_val2014_000000000042.jpg,42,What color are the gym shoes?,white,A curly-haired dog is sleeping on a shoe rack ...,A curly-haired dog is sleeping on a shoe rack ...,red
1,COCO_val2014_000000000073.jpg,73,What is the license number?,sv-6260,The motorcycle in the image has a license plat...,The motorcycle in the image has a license plat...,"{'answer': 'red', 'answer_confidence': 'no', '..."
2,COCO_val2014_000000000074.jpg,74,Does this dog have a collar?,no,The image shows a white dog sleeping on a cobb...,The image shows a white dog sleeping on a cobb...,red
3,COCO_val2014_000000000133.jpg,133,What color is lamp?,blue,The image shows a wooden loft bed with a small...,The image shows a wooden loft bed with a small...,red
4,COCO_val2014_000000000136.jpg,136,Is this in a museum?,no,The image shows two giraffes in an indoor zoo ...,The image shows two giraffes in a natural sava...,"{'answer': 'red', 'answer_confidence': 'no', '..."
...,...,...,...,...,...,...,...
995,COCO_val2014_000000014135.jpg,14135,Is it daytime?,yes,The image shows a skateboarder performing a tr...,The image shows a skateboarder performing a tr...,no
996,COCO_val2014_000000014151.jpg,14151,Is this at the Olympics?,yes,A ski jumper is in mid-air above a snow-covere...,A ski jumper is in mid-air above a snow-covere...,"No, this is not at the Olympics."
997,COCO_val2014_000000014167.jpg,14167,Is the skateboarder wearing safety gear?,no,The skateboarder is performing a trick down a ...,The skateboarder is performing a trick down a ...,"{'answer': 'blue', 'answer_confidence': 'yes',..."
998,COCO_val2014_000000014175.jpg,14175,What is sticking up from the fire hydrant?,nothing,The image shows a street scene with a fire hyd...,The image shows a street scene with a fire hyd...,red


In [39]:
df.to_csv('data/data_descriptions.csv')

In [ ]:
# preenchendo descrições faltantes

images_missing = df.loc[df['Tc'] == 'empty']
index_images_missing = df.loc[df['Tc'] == 'empty'].index

for i in range(images_missing.shape[0]):

    image_infos = df.iloc[index_images_missing[i]]
    image_question = image_infos['question']
    image_answer = image_infos['answer']
    image_path = base_image_path + image_infos['image_path']
    
    encoded_image = encode_image(image_path)

    prompt_formatado = PROMPT_CONSTRUCAO_TEXTO.format(question=image_question, answer=answer)

    print(f"Gerando textos para a imagem {i}.")

    response = model.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url" : {"url": f"data:image/jpeg;base64,{encoded_image}"}
                    },
                    {
                        "type": "text",
                        "text": prompt_formatado
                    }
                ]
            }
        ], temperature=0.0, timeout=20.0
    )
    texts = response.choices[0].message.content

    try:
        clean_text = texts.replace('\n', '')
        match = re.search(pattern, clean_text)

        if match:
            description1 = match.group(1).strip()
            description2 = match.group(2).strip()
            answer2 = match.group(3).strip()

            df.at[index_images_missing[i], 'Tc'] = description1
            df.at[index_images_missing[i], 'Tm'] = description2
            df.at[index_images_missing[i], 'Ans'] = answer2

        else:
            print(f'Resposta inesperada para a imagem no índice {index_images_missing[i]}: {clean_text}')
            df.at[index_images_missing[i], 'Tc'] = 'empty'
            df.at[index_images_missing[i], 'Tm'] = 'empty'
            df.at[index_images_missing[i], 'Ans'] = 'empty'

    except Exception as e:
        print(f'Erro ao carregar resposta: {e}')
        wrongs.append(generated_text)  

Gerando textos para a imagem 0.
Resposta inesperada para a imagem no índice 388: I'm sorry, I can't assist with that.
Gerando textos para a imagem 1.
Resposta inesperada para a imagem no índice 462: I'm sorry, I can't assist with that.
Gerando textos para a imagem 2.
Resposta inesperada para a imagem no índice 635: I'm sorry, I can't assist with that.
Gerando textos para a imagem 3.
Resposta inesperada para a imagem no índice 957: I'm sorry, I can't assist with that.


In [86]:
df

,image_path,image_id,question,answer,Tm,Tc,Ans
0,COCO_val2014_000000000042.jpg,42,What color are the gym shoes?,white,A curly-haired dog is sleeping on a shoe rack ...,A curly-haired dog is sleeping on a shoe rack ...,red
1,COCO_val2014_000000000073.jpg,73,What is the license number?,sv-6260,The motorcycle in the image has a license plat...,The motorcycle in the image has a license plat...,"{'answer': 'red', 'answer_confidence': 'no', '..."
2,COCO_val2014_000000000074.jpg,74,Does this dog have a collar?,no,The image shows a white dog sleeping on a cobb...,The image shows a white dog sleeping on a cobb...,red
3,COCO_val2014_000000000133.jpg,133,What color is lamp?,blue,The image shows a wooden loft bed with a small...,The image shows a wooden loft bed with a small...,red
4,COCO_val2014_000000000136.jpg,136,Is this in a museum?,no,The image shows two giraffes in an indoor zoo ...,The image shows two giraffes in a natural sava...,"{'answer': 'red', 'answer_confidence': 'no', '..."
...,...,...,...,...,...,...,...
995,COCO_val2014_000000014135.jpg,14135,Is it daytime?,yes,The image shows a skateboarder performing a tr...,The image shows a skateboarder performing a tr...,no
996,COCO_val2014_000000014151.jpg,14151,Is this at the Olympics?,yes,A ski jumper is in mid-air above a snow-covere...,A ski jumper is in mid-air above a snow-covere...,"No, this is not at the Olympics."
997,COCO_val2014_000000014167.jpg,14167,Is the skateboarder wearing safety gear?,no,The skateboarder is performing a trick down a ...,The skateboarder is performing a trick down a ...,"{'answer': 'blue', 'answer_confidence': 'yes',..."
998,COCO_val2014_000000014175.jpg,14175,What is sticking up from the fire hydrant?,nothing,The image shows a street scene with a fire hyd...,The image shows a street scene with a fire hyd...,red


In [ ]:
# verificando se ficou algum valor faltante (provavelmente pela imagem tratar de algum conteudo considerado sensivel)

df.loc[df['Tc'] == 'empty']

,image_path,image_id,question,answer,Tm,Tc,Ans
388,COCO_val2014_000000005503.jpg,5503,How many objects are below?,1,empty,empty,empty
462,COCO_val2014_000000006658.jpg,6658,Is this man naked?,yes,empty,empty,empty
635,COCO_val2014_000000008923.jpg,8923,What race is the boy in the picture?,caucasian,empty,empty,empty
957,COCO_val2014_000000013525.jpg,13525,What body part is visible?,back,empty,empty,empty


In [95]:
# removendo imagens onde as descrições foram nulas

final_df = df.loc[df['Tc'] != 'empty']

In [96]:
final_df

,image_path,image_id,question,answer,Tm,Tc,Ans
0,COCO_val2014_000000000042.jpg,42,What color are the gym shoes?,white,A curly-haired dog is sleeping on a shoe rack ...,A curly-haired dog is sleeping on a shoe rack ...,red
1,COCO_val2014_000000000073.jpg,73,What is the license number?,sv-6260,The motorcycle in the image has a license plat...,The motorcycle in the image has a license plat...,"{'answer': 'red', 'answer_confidence': 'no', '..."
2,COCO_val2014_000000000074.jpg,74,Does this dog have a collar?,no,The image shows a white dog sleeping on a cobb...,The image shows a white dog sleeping on a cobb...,red
3,COCO_val2014_000000000133.jpg,133,What color is lamp?,blue,The image shows a wooden loft bed with a small...,The image shows a wooden loft bed with a small...,red
4,COCO_val2014_000000000136.jpg,136,Is this in a museum?,no,The image shows two giraffes in an indoor zoo ...,The image shows two giraffes in a natural sava...,"{'answer': 'red', 'answer_confidence': 'no', '..."
...,...,...,...,...,...,...,...
995,COCO_val2014_000000014135.jpg,14135,Is it daytime?,yes,The image shows a skateboarder performing a tr...,The image shows a skateboarder performing a tr...,no
996,COCO_val2014_000000014151.jpg,14151,Is this at the Olympics?,yes,A ski jumper is in mid-air above a snow-covere...,A ski jumper is in mid-air above a snow-covere...,"No, this is not at the Olympics."
997,COCO_val2014_000000014167.jpg,14167,Is the skateboarder wearing safety gear?,no,The skateboarder is performing a trick down a ...,The skateboarder is performing a trick down a ...,"{'answer': 'blue', 'answer_confidence': 'yes',..."
998,COCO_val2014_000000014175.jpg,14175,What is sticking up from the fire hydrant?,nothing,The image shows a street scene with a fire hyd...,The image shows a street scene with a fire hyd...,red


In [97]:
final_df = final_df.reset_index(drop=True)

In [117]:
# tratando respostas que vieram como dicionário (o GPT deve ter respondido algumas dessa forma por já ter visto essa base muitas vezes, pois em nenhum momento esse formato foi passado)

for i in final_df.index:

    image_ans = final_df.iloc[i]['Ans']
    try:
        image_ans = image_ans.replace("'", '"')
        loaded_ans = json.loads(image_ans)
        if isinstance(loaded_ans, dict):
            ans = loaded_ans['answer']
            final_df.at[i, 'Ans'] = ans
            print(f'Resposta da imagem {i} convertida.')

    except json.JSONDecodeError as e:
        continue


Resposta da imagem 1 convertida.
Resposta da imagem 4 convertida.
Resposta da imagem 5 convertida.
Resposta da imagem 15 convertida.
Resposta da imagem 16 convertida.
Resposta da imagem 17 convertida.
Resposta da imagem 18 convertida.
Resposta da imagem 25 convertida.
Resposta da imagem 28 convertida.
Resposta da imagem 30 convertida.
Resposta da imagem 36 convertida.
Resposta da imagem 37 convertida.
Resposta da imagem 38 convertida.
Resposta da imagem 39 convertida.
Resposta da imagem 45 convertida.
Resposta da imagem 48 convertida.
Resposta da imagem 50 convertida.
Resposta da imagem 53 convertida.
Resposta da imagem 55 convertida.
Resposta da imagem 56 convertida.
Resposta da imagem 59 convertida.
Resposta da imagem 61 convertida.
Resposta da imagem 67 convertida.
Resposta da imagem 68 convertida.
Resposta da imagem 71 convertida.
Resposta da imagem 72 convertida.
Resposta da imagem 74 convertida.
Resposta da imagem 75 convertida.
Resposta da imagem 76 convertida.
Resposta da image

In [119]:
final_df

,image_path,image_id,question,answer,Tm,Tc,Ans
0,COCO_val2014_000000000042.jpg,42,What color are the gym shoes?,white,A curly-haired dog is sleeping on a shoe rack ...,A curly-haired dog is sleeping on a shoe rack ...,red
1,COCO_val2014_000000000073.jpg,73,What is the license number?,sv-6260,The motorcycle in the image has a license plat...,The motorcycle in the image has a license plat...,red
2,COCO_val2014_000000000074.jpg,74,Does this dog have a collar?,no,The image shows a white dog sleeping on a cobb...,The image shows a white dog sleeping on a cobb...,red
3,COCO_val2014_000000000133.jpg,133,What color is lamp?,blue,The image shows a wooden loft bed with a small...,The image shows a wooden loft bed with a small...,red
4,COCO_val2014_000000000136.jpg,136,Is this in a museum?,no,The image shows two giraffes in an indoor zoo ...,The image shows two giraffes in a natural sava...,red
...,...,...,...,...,...,...,...
991,COCO_val2014_000000014135.jpg,14135,Is it daytime?,yes,The image shows a skateboarder performing a tr...,The image shows a skateboarder performing a tr...,no
992,COCO_val2014_000000014151.jpg,14151,Is this at the Olympics?,yes,A ski jumper is in mid-air above a snow-covere...,A ski jumper is in mid-air above a snow-covere...,"No, this is not at the Olympics."
993,COCO_val2014_000000014167.jpg,14167,Is the skateboarder wearing safety gear?,no,The skateboarder is performing a trick down a ...,The skateboarder is performing a trick down a ...,blue
994,COCO_val2014_000000014175.jpg,14175,What is sticking up from the fire hydrant?,nothing,The image shows a street scene with a fire hyd...,The image shows a street scene with a fire hyd...,red


In [ ]:
# carregando dataset com textos irrelevantes utilizados no artigo

wikitext = pd.read_csv('data/WikiText_validation.csv')

In [18]:
wikitext

,Unnamed: 0,text
0,3,"Homarus gammarus , known as the European lobs..."
1,7,"Homarus gammarus is a large crustacean , with..."
2,8,The first pair of pereiopods is armed with a ...
3,9,"The exoskeleton is generally blue above , wit..."
4,10,The closest relative of H. gammarus is the Am...
...,...,...
1836,3739,"Throughout her career , Ayola has been outspo..."
1837,3743,Ayola was nominated and shortlisted for the '...
1838,3747,"Ayola describes herself as "" optimistic , cyn..."
1839,3748,"During a 1998 production of Hamlet , Ayola me..."


In [19]:
# pegando amostras aleatorias do wikitext para compor a coluna de texto irrelevante

random_samples_wikitext = wikitext["text"].sample(n=1000).tolist()

In [20]:
random_samples_wikitext

[' Several days after the low dissipated , the remnant moisture from Josephine brought showers and thunderstorms to St. Croix where up to 1 in ( 25 @.@ 4 mm ) of rain fell . The heavy rains led to minor street flooding and some urban flooding . No known damage was caused by the flood . \n',
 ' Kawaguchi \'s Center Body of troops was planning to attack the Lunga perimeter at the ridge , which they called " the centipede " ( <unk> gata ) because of its shape . On 9 September , Kawaguchi \'s troops left the coast at Koli Point . Split into four columns , they marched into the jungle towards their predesignated attack points south and southeast of the airfield . Lack of good maps , at least one faulty compass , and thick , almost impenetrable jungle caused the Japanese columns to proceed slowly and zigzag , costing a lot of time . At the same time , Oka \'s troops approached the Lunga perimeter from the west . Oka had some intelligence on the Marine defenses , extracted from a U.S. Army pi

In [21]:
final_df['Ti'] = pd.Series(random_samples_wikitext)

In [23]:
# salvando dados finais

final_df.to_csv('data/final_data_descriptions.csv')

## Generating Descriptions for Fine-Tuning

In [10]:
# carregando dataset com textos irrelevantes utilizados no artigo

wikitext_ft = pd.read_csv('data/WikiText_to_fine_tuning.csv')

In [11]:
wikitext_ft = wikitext_ft.sample(200).reset_index(drop=True)

In [12]:
wikitext_ft

,Unnamed: 0,text
0,245,"Upon commissioning , the sister ships were as..."
1,2905,"Returning , Nero sought for some place where ..."
2,2128,<formula> \n
3,344,"During the last week of August , US troops on..."
4,2184,"Coleman previously played Oswin Oswald in "" A..."
...,...,...
195,381,Liaison planes rose from the division strip e...
196,1985,Gorilla press slam \n
197,1812,"On 6 November , Édouard Adolphe Mortier comma..."
198,3156,Manila is politically divided into 896 barang...


In [8]:
dict_df_ft = {'image_path': [], 'image_id': [], 'question': [], 'answer': []}

for i, image in enumerate(os.listdir('images_fine_tuning')):
    image_id = get_image_id(image)
    print(f'Imagem {i}, ID: {image_id}')

    for annotation in json_annotations['annotations']:
        if image_id == str(annotation['image_id']):
            answers = annotation['answers']
            image_answers = []
            for answer in answers:
                image_answers.append(answer['answer'])

            chosen_answer = Counter(image_answers).most_common(1)[0][0] # obtendo a resposta que aparece mais vezes para ground-truth    

            question_id = annotation['question_id']
            for question in json_questions['questions']:
                if str(question_id) == str(question['question_id']):
                    chosen_question = question['question']
                    break

            dict_df_ft.get('image_id').append(image_id)
            dict_df_ft.get('question').append(chosen_question)
            dict_df_ft.get('answer').append(chosen_answer)
            dict_df_ft.get('image_path').append(image)
            break

Imagem 0, ID: 14248
Imagem 1, ID: 14257
Imagem 2, ID: 14265
Imagem 3, ID: 14271
Imagem 4, ID: 14276
Imagem 5, ID: 14278
Imagem 6, ID: 14282
Imagem 7, ID: 14285
Imagem 8, ID: 14297
Imagem 9, ID: 14306
Imagem 10, ID: 14321
Imagem 11, ID: 14338
Imagem 12, ID: 14352
Imagem 13, ID: 14353
Imagem 14, ID: 14380
Imagem 15, ID: 14392
Imagem 16, ID: 14428
Imagem 17, ID: 14439
Imagem 18, ID: 14450
Imagem 19, ID: 14470
Imagem 20, ID: 14473
Imagem 21, ID: 14494
Imagem 22, ID: 14504
Imagem 23, ID: 14526
Imagem 24, ID: 14547
Imagem 25, ID: 14549
Imagem 26, ID: 14557
Imagem 27, ID: 14562
Imagem 28, ID: 14567
Imagem 29, ID: 14591
Imagem 30, ID: 14592
Imagem 31, ID: 14600
Imagem 32, ID: 14629
Imagem 33, ID: 14635
Imagem 34, ID: 14681
Imagem 35, ID: 14713
Imagem 36, ID: 14723
Imagem 37, ID: 14733
Imagem 38, ID: 14743
Imagem 39, ID: 14756
Imagem 40, ID: 14773
Imagem 41, ID: 14781
Imagem 42, ID: 14820
Imagem 43, ID: 14821
Imagem 44, ID: 14831
Imagem 45, ID: 14845
Imagem 46, ID: 14869
Imagem 47, ID: 14874
Im

In [9]:
df_ft = pd.DataFrame(dict_df_ft)

In [10]:
# dividindo as 1000 amostras em 5 categorias, assim como feito pelo artigo
df_ft['type'] = ''
df_ft.loc[:200, 'type'] = 'text only'
df_ft.loc[200:400, 'type'] = 'original VQA'
df_ft.loc[400:600, 'type'] = 'match'
df_ft.loc[600:800, 'type'] = 'corrupted'
df_ft.loc[800:, 'type'] = 'irrelevant'


In [11]:
# criando colunas para preencher texto certo e corrompido nas amostras necessárias

df_ft['Tm'] = ''
df_ft['Tc'] = ''
df_ft['Ti'] = ''

In [12]:
df_ft

,image_path,image_id,question,answer,type,Tm,Tc,Ti
0,COCO_val2014_000000014248.jpg,14248,What is laying on the bed?,girl,text only,,,
1,COCO_val2014_000000014257.jpg,14257,Are the two people friends?,yes,text only,,,
2,COCO_val2014_000000014265.jpg,14265,Is the man carrying a bag?,yes,text only,,,
3,COCO_val2014_000000014271.jpg,14271,What kind is the tree?,oak,text only,,,
4,COCO_val2014_000000014276.jpg,14276,What is the wall made of?,trees,text only,,,
...,...,...,...,...,...,...,...,...
995,COCO_val2014_000000027932.jpg,27932,Who does the equipment belong too?,baseball player,irrelevant,,,
996,COCO_val2014_000000027935.jpg,27935,How many flowers are in each bucket?,lot,irrelevant,,,
997,COCO_val2014_000000027946.jpg,27946,Urban or suburban?,urban,irrelevant,,,
998,COCO_val2014_000000027969.jpg,27969,What is behind the woman?,parking lot,irrelevant,,,


In [22]:
def generating_descriptions_ft(df: pd.DataFrame, model):

    base_image_path = 'images_fine_tuning/'
    df = df.copy()

    for i in df.index:

        image_infos = df.iloc[i]
        if image_infos['type'] == 'match' or image_infos['type'] == 'corrupted':

            image_question = image_infos['question']
            image_answer = image_infos['answer']
            image_path = base_image_path + image_infos['image_path']
            
            encoded_image = encode_image(image_path)

            prompt_formatado = PROMPT_CONSTRUCAO_TEXTO.format(question=image_question, answer=image_answer)

            print(f"Gerando textos para a imagem {i}.")

            response = model.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "image_url",
                                "image_url" : {"url": f"data:image/jpeg;base64,{encoded_image}"}
                            },
                            {
                                "type": "text",
                                "text": prompt_formatado
                            }
                        ]
                    }
                ], temperature=0.0, timeout=20.0
            )
            text = response.choices[0].message.content

            pattern = r"Description 1:\s*(.*?)Description 2:\s*(.*?)Answer 2:\s*(.*)"
            try:
                clean_text = text.replace('\n', '')
                match = re.search(pattern, clean_text)

                if match:
                    description1 = match.group(1).strip()
                    description2 = match.group(2).strip()

                    if image_infos['type'] == 'match':
                        df.at[i, 'Tm'] = description1
                        tm = df.iloc[i]['Tm']
                        print(f'Descrição correta gerada: {tm}')
                    
                    elif image_infos['type'] == 'corrupted':
                        df.at[i, 'Tc'] = description2
                        tc = df.iloc[i]['Tc']
                        print(f'Descrição corrompida gerada: {tc}')

                else:
                    print(f'Resposta inesperada para a imagem {i}: {clean_text}')        

            except Exception as e:
                print(f'Erro ao carregar resposta: {e}')

    return df


In [23]:
model = OpenAI(api_key=OPENAI_KEY)

In [24]:
final_df_ft = generating_descriptions_ft(df_ft, model)

Gerando textos para a imagem 400.
Descrição correta gerada: Three players are on a grassy field, two in white uniforms and one in a blue uniform, actively engaged with a soccer ball.
Gerando textos para a imagem 401.
Descrição correta gerada: A group of people is standing outside, and one person is wearing a red shirt.
Gerando textos para a imagem 402.
Descrição correta gerada: The man is standing in a spacious area with a glass ceiling and walls, typical of an atrium.
Gerando textos para a imagem 403.
Descrição correta gerada: The meal is served on a wooden table with natural lighting, suggesting an outdoor setting.
Gerando textos para a imagem 404.
Descrição correta gerada: The vase features an artistic depiction of a person in a curled position, with intricate details visible on its surface.
Gerando textos para a imagem 405.
Descrição correta gerada: The table has a glass filled with a beverage next to a slice of deep-dish pizza.
Gerando textos para a imagem 406.
Descrição correta g

In [22]:
for idx, i in enumerate(final_df_ft.index[800:]):
    
    final_df_ft.at[i, 'Ti'] = wikitext_ft.iloc[idx]['text']

In [34]:
final_df_ft

,image_path,image_id,question,answer,type,Tm,Tc,Ti
0,COCO_val2014_000000014248.jpg,14248,What is laying on the bed?,girl,text only,,,
1,COCO_val2014_000000014257.jpg,14257,Are the two people friends?,yes,text only,,,
2,COCO_val2014_000000014265.jpg,14265,Is the man carrying a bag?,yes,text only,,,
3,COCO_val2014_000000014271.jpg,14271,What kind is the tree?,oak,text only,,,
4,COCO_val2014_000000014276.jpg,14276,What is the wall made of?,trees,text only,,,
...,...,...,...,...,...,...,...,...
995,COCO_val2014_000000027932.jpg,27932,Who does the equipment belong too?,baseball player,irrelevant,,,Liaison planes rose from the division strip e...
996,COCO_val2014_000000027935.jpg,27935,How many flowers are in each bucket?,lot,irrelevant,,,Gorilla press slam \n
997,COCO_val2014_000000027946.jpg,27946,Urban or suburban?,urban,irrelevant,,,"On 6 November , Édouard Adolphe Mortier comma..."
998,COCO_val2014_000000027969.jpg,27969,What is behind the woman?,parking lot,irrelevant,,,Manila is politically divided into 896 barang...


In [33]:
final_df_ft.to_csv('data/final_df_ft.csv')